In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import Compose, ToTensor, Lambda, Resize, Normalize
from PIL import Image, ImageDraw
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score,f1_score

from tqdm import tqdm
from transformers import ViTForImageClassification, TrainingArguments, Trainer

## Finetuning Vision Transformer (ViT) for Image Classification

### Parameter

In [2]:
DIRECTROY = 'data'
MODEL_PATH = 'models'
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LR = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
df_train = pd.read_csv(f'{DIRECTROY}/reduced_train.csv') 
df_test = pd.read_csv(f'{DIRECTROY}/reduced_test.csv') 
num_classes = len(df_train['newid'].unique())

In [5]:
df_test_public = df_test[df_test['Usage'] == 'Public']
df_test_private = df_test[df_test['Usage'] == 'Private']

In [6]:
len(df_test_public), len(df_test_private)

(2026, 4052)

# Divide the train into multiple chunks

Due to lack of ram, i will have to divide the dataloader into multiple dataloader and save it in SSD.

In [7]:
image_transforms = Compose([
    Resize((IMG_SIZE, IMG_SIZE)),
    ToTensor(), 
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
])

In [8]:
from transformers import CLIPTokenizerFast

### Custom Dataset
Creating image with border like in the EDA

In [9]:
class CustomDataset(Dataset):
    def __init__(self, df, transforms, directory):
        self.tokenizer =  CLIPTokenizerFast.from_pretrained("openai/clip-vit-base-patch16")
        self.df = df
        self.transforms = transforms
        self.directory = directory
        self.labels = torch.Tensor(df['newid'].values).long()
        self.imgs = torch.cat([ self.transforms(self.resize_img(Image.open(f'{DIRECTROY}/{self.directory}/{x}')).convert('RGB')).half().reshape(1,3,IMG_SIZE,IMG_SIZE) for x in tqdm(df['name'].values)])
        self.tokenized = self.tokenizer(df['label'].tolist(), padding=True, truncation=True, return_tensors="pt")
        self.input_ids = self.tokenized['input_ids']
        self.attention_mask = self.tokenized['attention_mask']
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        return img, label, input_ids, attention_mask
    
    @staticmethod
    def resize_img(a):
        w, h = a.size
        if w < h:
            scale = h/IMG_SIZE
            w = int(w/scale)
            h = IMG_SIZE
            a = a.resize((w, h))
            lb= np.array([a.load()[0,x] for x in range(h)])
            rb = np.array([a.load()[w-1,x] for x in range(h)])
            lb = lb.mean(axis=0).astype('uint8')
            rb = rb.mean(axis=0).astype('uint8')
            pic = Image.new('RGB', (h, h), color = (255, 255, 255))
            imgl = Image.new('RGB', (h//2, h), color = tuple(lb))
            imgr = Image.new('RGB', (h//2, h), color = tuple(rb))
            
            pic.paste(imgl, (0, 0))
            pic.paste(imgr, (h//2, 0))
            
            pic.paste(a, (h//2-w//2, 0))

        elif w>h:
            scale = w/IMG_SIZE
            h = int(h/scale)
            w = IMG_SIZE
            a = a.resize((w, h))
            
            lb= np.array([a.load()[x,0] for x in range(w)])
            rb = np.array([a.load()[x,h-1] for x in range(w)])
            lb = lb.mean(axis=0).astype('uint8')
            rb = rb.mean(axis=0).astype('uint8')
            
            pic = Image.new('RGB', (w, w), color = (255, 255, 255))
            imgl = Image.new('RGB', (w, w//2), color = tuple(lb))
            imgr = Image.new('RGB', (w, w//2), color = tuple(rb))
            
            pic.paste(imgl, (0, 0))
            pic.paste(imgr, (0, w//2))
            
            pic.paste(a, (0, w//2-h//2))

        else:
            a = a.resize((IMG_SIZE, IMG_SIZE))
            pic = a
        return pic

In [10]:
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test_public = df_test_public.sample(frac=1).reset_index(drop=True)
df_test_private = df_test_private.sample(frac=1).reset_index(drop=True)

In [11]:
len(df_train['newid'].unique()), len(df_test['newid'].unique())

(649, 649)

In [12]:
df_train.head()

,name,class,group,label,newid
0,86940.jpg,6048,195,warship leog set,330
1,10895.jpg,1363,23,blue workout short,90
2,49510.jpg,4164,145,white earphones,364
3,40239.jpg,3735,60,pink luggage with vertical stripes,638
4,8492.jpg,1107,23,black jogging trousers with white stripe,21


In [37]:
import math

In [13]:
train_dataset = CustomDataset(df_train, image_transforms, 'train')
torch.save(train_dataset, f'{DIRECTROY}/train_dataset/train_dataset_reduced_prompts.pth')

100%|██████████| 29243/29243 [06:16<00:00, 77.66it/s]


In [38]:
for i in range(math.ceil(len(df_train)/18192)):
    train_dataset = CustomDataset(df_train[i*18192:(i+1)*18192], image_transforms, 'train')
    torch.save(train_dataset, f'{DIRECTROY}/train_dataset/train_dataset_reduced_{i}.pth')
    del train_dataset

100%|██████████| 11051/11051 [02:23<00:00, 76.78it/s]


In [39]:
for i in range(math.ceil(len(df_test)/18192)):
    test_dataset = CustomDataset(df_test[i*18192:(i+1)*18192], image_transforms, 'test')
    torch.save(test_dataset, f'{DIRECTROY}/test_public_dataset/test_public_reduced_dataset_{i}.pth')
    del test_dataset

# for i in range(math.ceil(len(df_test_private)/16192)):
#     test_dataset = CustomDataset(df_test_private[i*16192:(i+1)*16192], image_transforms, 'test')
#     torch.save(test_dataset, f'{DIRECTROY}/test_private_dataset/test_private_reduced_dataset_{i}.pth')
#     del test_dataset


100%|██████████| 6078/6078 [01:08<00:00, 88.28it/s] 


### Load ViT

In [40]:
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=num_classes)
model = model.to(device)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
model.parameters

<bound method Module.parameters of ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
      

### Training

In [42]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = LR)
scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=EPOCHS)

In [44]:
max_accuracy = 0.0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    
    # Training loop
    print('Training epoch:', epoch+1)
    len_train = 0
    for i in range(2):
        try:
            train_dataset = torch.load(f'{DIRECTROY}/train_dataset/train_dataset_reduced_{i}.pth')
            train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        except:
            break
        
        for inputs, labels, input_ids, attention_mask in tqdm(train_dataloader, desc=f'Batch {i+1}/{2}'):
            optimizer.zero_grad()
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs.logits, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        len_train += len(train_dataset)
        del train_dataset
        
    scheduler.step()    
    train_loss/=len_train
    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {train_loss}')
    
    eval_loss = 0.0
    model.eval()
    
    true_labels = []
    pred_labels = []
    
    print('Evaluating epoch:', epoch+1)
    with torch.no_grad():
        len_test = 0
        for i in range(1):
            try:
                test_dataset = torch.load(f'{DIRECTROY}/test_public_dataset/test_public_reduced_dataset_{i}.pth')
                test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
            except:
                break

            for inputs, labels, input_ids, attention_mask in tqdm(test_dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs.logits, labels).to(device)
                eval_loss += loss.item()
                
                outputs = torch.argmax(outputs.logits, 1).flatten().cpu().numpy()
                labels = labels.flatten().cpu().numpy()
                
                true_labels.extend(labels)
                pred_labels.extend(outputs)
            
            len_test += len(test_dataset)
            del test_dataset
        
        print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {eval_loss/len_test}')
        print(f'Accuracy: {accuracy_score(true_labels, pred_labels)}')
        print(f'F1 Score Weighted: {f1_score(true_labels, pred_labels, average="weighted")}')
        print(f'F1 Score Macro: {f1_score(true_labels, pred_labels, average="macro")}')
        if accuracy_score(true_labels, pred_labels) > max_accuracy:
            max_accuracy = accuracy_score(true_labels, pred_labels)
            torch.save(model.state_dict(), f'{MODEL_PATH}/vit_reduced_model_{epoch+1}.pth')
            torch.save(optimizer.state_dict(), f'{MODEL_PATH}/optimizer/vit_reduced_optimizer_{epoch+1}.pth')
            

Training epoch: 1


Batch 2/2: 100%|██████████| 346/346 [01:24<00:00,  4.10it/s]


Epoch 1/50, Loss: 0.14573990254682134
Evaluating epoch: 1


100%|██████████| 190/190 [00:17<00:00, 11.15it/s]


Epoch 1/50, Loss: 0.09127754226018034
Accuracy: 0.5378413951957881
F1 Score Weighted: 0.45008926881853034
F1 Score Macro: 0.38259801624946366
Training epoch: 2


...

Epoch 29/50, Loss: 0.00024532172882561195
Evaluating epoch: 29


100%|██████████| 190/190 [00:16<00:00, 11.22it/s]


Epoch 29/50, Loss: 0.022219705306231137
Accuracy: 0.8538993089832182
F1 Score Weighted: 0.85176557305243
F1 Score Macro: 0.8508067435974935
Training epoch: 30


Batch 1/2:  31%|███       | 177/569 [00:42<01:34,  4.14it/s]


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('models/vit_reduced_model_23.pth'))

<All keys matched successfully>

In [ ]:
with torch.no_grad():
    true_labels = []
    pred_labels = []
    len_test = 0
    for i in range(1):
        try:
            test_dataset = torch.load(f'{DIRECTROY}/test_public_dataset/test_public_reduced_dataset_{i}.pth')
            test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
        except:
            break

        for inputs, labels in tqdm(test_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs.logits, labels).to(device)
            eval_loss += loss.item()
            
            outputs = torch.argmax(outputs.logits, 1).flatten().cpu().numpy()
            labels = labels.flatten().cpu().numpy()
            
            true_labels.extend(labels)
            pred_labels.extend(outputs)
        
        len_test += len(test_dataset)
        del test_dataset
    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {eval_loss/len_test}')
    print(f'Accuracy: {accuracy_score(true_labels, pred_labels)}')
    print(f'F1 Score Weighted: {f1_score(true_labels, pred_labels, average="weighted")}')
    print(f'F1 Score Macro: {f1_score(true_labels, pred_labels, average="macro")}')

100%|██████████| 186/186 [00:16<00:00, 10.98it/s]

Epoch 93/100, Loss: 0.0652661323842611
Accuracy: 0.7968382105617222
F1 Score Weighted: 0.7896039842586119
F1 Score Macro: 0.7896039842586119
